# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install azureml-widgets -q
%pip install --upgrade azureml-sdk[notebooks,automl] -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-ml 0.6.1 requires enum34, which is not installed.
scikit-image 0.24.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
dask-expr 1.1.14 requires dask==2024.9.0, but you have dask 2023.2.0 which is incompatible.
dask-expr 1.1.14 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
azureml-train 1.57.0 requires azureml-train-core~=1.57.0, but you have azureml-train-core 1.58.0 which is incompatible.
azureml-tensorboard 1.57.0 require

In [23]:
import os
import csv
import logging
import json
import requests 

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from matplotlib import pyplot as plt

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget



## Create Workspace

In [4]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-269850
Azure region: westeurope
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-269850


## Create Experiment

In [5]:
# choose a name for experiment
experiment_name = 'azmlproject3'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
azmlproject3,quick-starts-ws-269850,Link to Azure Machine Learning studio,Link to Documentation


## Create compute cluster

In [8]:
# name compute cluster
amlcompute_cluster_name = "azmlproject3"

# check use compute cluster if exist, else create
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Not found existed compute cluster, creating one...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D3_v2',
                                                           min_nodes=1,
                                                           max_nodes=72)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    print('Create new compute cluster successfully')

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

In [9]:
found = False
key = "credit-default"
description_text = "credit card default dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data_url = 'https://raw.githubusercontent.com/TungPhamDuy/azmlproject3/refs/heads/main/credit-card-default.csv'
        dataset = Dataset.Tabular.from_delimited_files(data_url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head(5)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,id,linitBal,sex,education,married,age,pay_0,pay_2,pay_3,pay_4,...,billAmt_4,billAmt_5,billAmt_6,payAmt_1,payAmt_2,payAmt_3,payAmt_4,payAmt_5,payAmt_6,defaulted
0,1,20000,0,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,0,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,0,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,0,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
automl_settings = {
    "experiment_timeout_hours": 0.5,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "iterations": 20
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="defaulted",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
# TODO: Submit your experiment and see widget
automl_run = experiment.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on azmlproject3 with default configuration
Running on remote compute: azmlproject3


Experiment,Id,Type,Status,Details Page,Docs Page
azmlproject3,AutoML_ef9af502-dc65-4305-9842-ee069956b899,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
# Retrieve and save your best automl model.
best, model = automl_run.get_output()
best

Experiment,Id,Type,Status,Details Page,Docs Page
azmlproject3,AutoML_ef9af502-dc65-4305-9842-ee069956b899_18,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
model

Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/azmlproject3/code/Users/odl_user_269850')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('11', Pipeline...r', Normalizer()), ('randomforestclassifier', RandomForestClassifier(class_weight='balanced', min_samples_leaf=0.01, min_samples_split=0.01, n_jobs=1, oob_score=True))]))], flatten_transform=False, weights=[0.2, 0.06666666666666667, 0.13333333333333333, 0.2, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667]))])

In [17]:
registered_model = best.register_model(
    model_path="outputs/model.pkl",
    model_name="best_automl",
    properties={"accuracy": best.get_metrics()["accuracy"]}
)

In [15]:
best.get_metrics()

{'recall_score_weighted': 0.8894512605953251,
 'accuracy': 0.8894512605953251,
 'average_precision_score_micro': 0.9531308097183683,
 'AUC_macro': 0.9423344919054059,
 'f1_score_micro': 0.8894512605953251,
 'recall_score_micro': 0.8894512605953251,
 'f1_score_macro': 0.8890948299050544,
 'precision_score_weighted': 0.8940246255387084,
 'matthews_correlation': 0.7822208029692302,
 'AUC_micro': 0.9522585363318199,
 'average_precision_score_weighted': 0.935297778983947,
 'f1_score_weighted': 0.8897399371950033,
 'precision_score_macro': 0.889086047028548,
 'precision_score_micro': 0.8894512605953251,
 'balanced_accuracy': 0.8931467167749577,
 'average_precision_score_macro': 0.9319989791191219,
 'recall_score_macro': 0.8931467167749577,
 'weighted_accuracy': 0.8858415773590127,
 'AUC_weighted': 0.9423344919054059,
 'log_loss': 0.36114579294152405,
 'norm_macro_recall': 0.7862934335499155,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_ef9af502-dc65-4305-9842-ee069956b899_1

In [18]:
best.download_file('outputs/conda_env_v_1_0_0.yml','conda_env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
best.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [37]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

env = best.get_environment()
infer_config = InferenceConfig(entry_script = 'scoring.py', environment = env)

# deloyment config
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 2, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)
# model deloy
service = Model.deploy(ws, "deloy-best-model1", 
                       [registered_model], 
                       infer_config, 
                       deployment_config)
# wait for deloyment
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-11-17 15:54:16+00:00 Creating Container Registry if not exists.
2024-11-17 15:54:18+00:00 Use the existing image.
2024-11-17 15:54:19+00:00 Submitting deployment to compute.
2024-11-17 15:54:24+00:00 Checking the status of deployment deloy-best-model1..
2024-11-17 15:58:50+00:00 Checking the status of inference endpoint deloy-best-model1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [41]:
# get 5 data from the df to test 
df_test = df.sample(5)
df_labeled = df_test.pop('defaulted')

# turn data to json format
send_request = json.dumps({'data': df_test.to_dict(orient='records')})
print(send_request)

{"data": [{"id": 7247, "linitBal": 230000, "sex": 1, "education": 1, "married": 1, "age": 32, "pay_0": 1, "pay_2": -2, "pay_3": -2, "pay_4": -2, "pay_5": -2, "pay_6": -1, "billAmt_1": 0, "billAmt_2": 0, "billAmt_3": 0, "billAmt_4": 0, "billAmt_5": 0, "billAmt_6": 150, "payAmt_1": 0, "payAmt_2": 0, "payAmt_3": 0, "payAmt_4": 0, "payAmt_5": 150, "payAmt_6": 990}, {"id": 8115, "linitBal": 90000, "sex": 0, "education": 2, "married": 2, "age": 28, "pay_0": 2, "pay_2": 0, "pay_3": 0, "pay_4": 0, "pay_5": 2, "pay_6": 0, "billAmt_1": 48177, "billAmt_2": 49563, "billAmt_3": 47750, "billAmt_4": 47169, "billAmt_5": 46284, "billAmt_6": 47519, "payAmt_1": 2500, "payAmt_2": 2500, "payAmt_3": 3700, "payAmt_4": 0, "payAmt_5": 2200, "payAmt_6": 1800}, {"id": 5309, "linitBal": 80000, "sex": 1, "education": 1, "married": 2, "age": 35, "pay_0": 0, "pay_2": 0, "pay_3": 0, "pay_4": 0, "pay_5": 0, "pay_6": 0, "billAmt_1": 52065, "billAmt_2": 55205, "billAmt_3": 55900, "billAmt_4": 55622, "billAmt_5": 56295, 

In [42]:
auth_key = service.get_keys()[0]
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {auth_key}'
model_response = requests.post(service.scoring_uri, send_request, headers=headers)

In [43]:
print('Expected actual output:', list(df_labeled))
print('Response from model:', model_response.text)

Expected actual output: [1, 1, 0, 1, 0]
Response from model: "{\"result\": [1, 1, 0, 1, 0]}"


In [44]:
service.get_logs()

'2024-11-17T15:58:43,287922800+00:00 - rsyslog/run \n2024-11-17T15:58:43,296963000+00:00 - gunicorn/run \n2024-11-17T15:58:43,298787800+00:00 | gunicorn/run | \n2024-11-17T15:58:43,300888500+00:00 | gunicorn/run | ###############################################\n2024-11-17T15:58:43,306930000+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-11-17T15:58:43,308364900+00:00 | gunicorn/run | ###############################################\n2024-11-17T15:58:43,310240400+00:00 | gunicorn/run | \n2024-11-17T15:58:43,323397900+00:00 | gunicorn/run | \n2024-11-17T15:58:43,325878600+00:00 - nginx/run \n2024-11-17T15:58:43,333347800+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1\n2024-11-17T15:58:43,334940900+00:00 | gunicorn/run | \n2024-11-17T15:58:43,340959600+00:00 | gunicorn/run | \n2024-11-17T15:58:43,344047200+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

In [45]:
service.delete()

Running
2024-11-17 16:24:24+00:00 Deleting service.
2024-11-17 16:24:26+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
